# Исследование моделей обнаружения и распознавания объектов с использованием Ultralytics (YOLOv11)

# 📌 Установка необходимых библиотек

In [1]:
!pip install ultralytics
!pip install roboflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.5/983.5 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

# 📌 Подключение к Google Drive и распаковка архива

In [2]:
from google.colab import drive
import zipfile, os, shutil
import os
from shutil import move

drive.mount('/content/drive')

# Пути
zip_path = '/content/drive/MyDrive/Colab Notebooks/AiDatasets/smokedata.zip'
extract_path = '/content/smokedata_raw'
final_path = '/content/smokedata'

# Распаковываем
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Создаем финальную структуру директорий
for subset in ['train', 'val', 'test']:
    os.makedirs(os.path.join(final_path, 'images', subset), exist_ok=True)
    os.makedirs(os.path.join(final_path, 'labels', subset), exist_ok=True)

# Функция для перемещения изображений и меток
def move_data(src_folder, dst_img_folder, dst_label_folder):
    for fname in os.listdir(src_folder):
        full_path = os.path.join(src_folder, fname)
        if fname.endswith(('.jpg', '.jpeg', '.png')):
            shutil.move(full_path, os.path.join(dst_img_folder, fname))
        elif fname.endswith('.txt'):
            shutil.move(full_path, os.path.join(dst_label_folder, fname))

# Переносим данные из исходных папок
move_data(os.path.join(extract_path, 'Training', 'Training'),
          os.path.join(final_path, 'images', 'train'),
          os.path.join(final_path, 'labels', 'train'))

move_data(os.path.join(extract_path, 'Validation', 'Validation'),
          os.path.join(final_path, 'images', 'val'),
          os.path.join(final_path, 'labels', 'val'))

move_data(os.path.join(extract_path, 'Testing', 'Testing'),
          os.path.join(final_path, 'images', 'test'),
          os.path.join(final_path, 'labels', 'test'))

Mounted at /content/drive


In [3]:
import zipfile
import os
# Destination directory
extract_dir = 'smokedata_raw'

# Create the destination directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Extract all contents of the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Archive '{zip_path}' successfully extracted to '{extract_dir}'")

Archive '/content/drive/MyDrive/Colab Notebooks/AiDatasets/smokedata.zip' successfully extracted to 'smokedata_raw'


# 1. Выбор начальных условий
# a. Выбор набора данных
#   Выбран набор данных курящих и некурящих людей. Это имеет прикладное значение, например, для мониторинга нарушений правил в общественных местах или для медицинских анализов.
# b. Метрики качества
#   Используемые метрики: mAP@0.5, precision, recall. Они лучше всего подходят для оценки моделей детекции объектов.

# 📌 Импорт и подготовка

In [4]:
from ultralytics import YOLO
from sklearn.metrics import classification_report
import shutil

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# 📌 Подготовка данных в формате YOLO
# Папки должны быть следующего вида:
# smokedata/
# ├── images/
# │   ├── train/
# │   ├── val/
# │   └── test/
# └── labels/
#     ├── train/
#     ├── val/
#     └── test/

# 🧱 Создание структуры для YOLO

# 📄 Создание файла конфигурации данных для обучения

In [5]:
with open("/content/smokedata/data.yaml", "w") as f:
    f.write("""
path: /content/smokedata
train: images/train
val: images/val
test: images/test
nc: 2
names: ['non-smoker', 'smoker']
""")


# 2. Создание бейзлайна и оценка качества
# a. Обучение моделей из семейства YOLOv11 (YOLOv8 в ultralytics)

# Обучение сверточной модели YOLOv8n (наиболее легкая для Colab)

In [6]:
model_cnn = YOLO('yolov8n.pt')
model_cnn.train(data='/content/smokedata/data.yaml', epochs=3, imgsz=640, batch=16, name='baseline_cnn')

100%|██████████| 6.25M/6.25M [00:00<00:00, 113MB/s]


Ultralytics 8.3.114 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/smokedata/data.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=baseline_cnn, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, lin

100%|██████████| 755k/755k [00:00<00:00, 26.6MB/s]

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

Model summary: 129 layers, 3,011,238 parameters, 3,011,222 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:04<00:00,  1.47it/s]
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:491: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:526: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:526: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_met

                   all        180          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:655: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index


Speed: 0.3ms preprocess, 3.6ms inference, 0.0ms loss, 12.4ms postprocess per image
Results saved to runs/detect/baseline_cnn


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([], dtype=int64)
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7be99769be10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

In [7]:
!cat /content/smokedata/data.yaml


path: /content/smokedata
train: images/train
val: images/val
test: images/test
nc: 2
names: ['non-smoker', 'smoker']


# Обучение трансформерной модели YOLOv8x (условно трансформерная архитектура)

In [8]:
import os
import cv2
import numpy as np
from glob import glob

base_path = '/content/smokedata'
splits = ['train', 'val', 'test']
CLASS_ID = 0  # Предположим, у нас один класс — дым

def create_yolo_label(mask_path, label_path, img_shape):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    contours, _ = cv2.findContours((mask > 127).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    h, w = img_shape[:2]
    lines = []

    for contour in contours:
        x, y, bw, bh = cv2.boundingRect(contour)
        # YOLO format
        x_c = (x + bw / 2) / w
        y_c = (y + bh / 2) / h
        bw /= w
        bh /= h
        lines.append(f"{CLASS_ID} {x_c:.6f} {y_c:.6f} {bw:.6f} {bh:.6f}")

    with open(label_path, 'w') as f:
        f.write("\n".join(lines))

for split in splits:
    image_dir = os.path.join(base_path, 'images', split)
    mask_dir = os.path.join(base_path, 'masks', split)
    label_dir = os.path.join(base_path, 'labels', split)
    os.makedirs(label_dir, exist_ok=True)

    for mask_path in glob(f"{mask_dir}/*.png"):
        file_stem = os.path.basename(mask_path).replace('.png', '')  # пример: 'img001'
        image_path = os.path.join(image_dir, f"{file_stem}.jpg")
        label_path = os.path.join(label_dir, f"{file_stem}.txt")

        if not os.path.exists(image_path):
            print(f"⚠️ Нет изображения для маски: {image_path}")
            continue

        img = cv2.imread(image_path)
        create_yolo_label(mask_path, label_path, img.shape)
        print(f"✅ Аннотация создана: {label_path}")


In [9]:
import glob

# Выведем все доступные изображения в папке валидации
images = glob.glob('/content/smokedata/labels/val/*')
print("Найдено изображений:", len(images))
for path in images[:5]:  # покажем первые 5
    print(path)


Найдено изображений: 0


In [10]:
!find /content/smokedata/labels/val -name "*.txt" -exec grep . {} \;

In [11]:
pip install ultralytics

In [12]:
model = YOLO('yolov8x.pt')

# Папка с изображениями, которые хотим разметить
image_dir = '/content/smokedata/images/train'  # или 'val', 'test' по нужде
label_dir = '/content/smokedata/labels/train'
os.makedirs(label_dir, exist_ok=True)

# Получаем список изображений
image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))]

for img_path in image_paths:
    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    # Получаем предсказания
    results = model(img, verbose=False)[0]

    txt_path = os.path.join(label_dir, os.path.splitext(os.path.basename(img_path))[0] + '.txt')

    with open(txt_path, 'w') as f:
        for box in results.boxes:
            cls = int(box.cls[0])
            x1, y1, x2, y2 = box.xyxy[0]

            # YOLO формат (x_center, y_center, width, height), всё в относительных координатах [0..1]
            x_center = ((x1 + x2) / 2) / w
            y_center = ((y1 + y2) / 2) / h
            bw = (x2 - x1) / w
            bh = (y2 - y1) / h

            f.write(f"{cls} {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f}\n")

100%|██████████| 131M/131M [00:01<00:00, 92.7MB/s]


In [13]:
model_transformer = YOLO('yolov8x.pt')
model_transformer.train(data='/content/smokedata/data.yaml', epochs=3, imgsz=640, batch=8, name='baseline_transformer')

Ultralytics 8.3.114 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/content/smokedata/data.yaml, epochs=3, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=baseline_transformer, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=Tr

train: Scanning /content/smokedata/labels/train... 716 images, 5 backgrounds, 381 corrupt: 100%|██████████| 716/716 [00:00<00:00, 1066.50it/s]

train: /content/smokedata/images/train/notsmoking_0001.jpg: ignoring corrupt image/label: Label class 32 exceeds dataset class count 2. Possible class labels are 0-1
train: /content/smokedata/images/train/notsmoking_0010.jpg: ignoring corrupt image/label: Label class 27 exceeds dataset class count 2. Possible class labels are 0-1
train: /content/smokedata/images/train/notsmoking_0011.jpg: ignoring corrupt image/label: Label class 74 exceeds dataset class count 2. Possible class labels are 0-1
train: /content/smokedata/images/train/notsmoking_0019.jpg: ignoring corrupt image/label: Label class 27 exceeds dataset class count 2. Possible class labels are 0-1
train: /content/smokedata/images/train/notsmoking_0022.jpg: ignoring corrupt image/label: Label class 66 exceeds dataset class count 2. Possible class labels are 0-1
train: /content/smokedata/images/train/notsmoking_0025.jpg: ignoring corrupt image/label: Label class 59 exceeds dataset class count 2. Possible class labels are 0-1
trai

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 502.4±90.6 MB/s, size: 75.6 KB)


val: Scanning /content/smokedata/labels/val.cache... 0 images, 180 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:00<?, ?it/s]

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:05<00:00,  2.06it/s]
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:491: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:526: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:526: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_m

                   all        180          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:655: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index


Speed: 0.3ms preprocess, 26.6ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to runs/detect/baseline_transformer


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([], dtype=int64)
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7be98e18fb50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

# b. Оценка моделей

In [14]:
metrics_cnn = model_cnn.val()
metrics_transformer = model_transformer.val()

Ultralytics 8.3.114 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2386.2±686.8 MB/s, size: 81.9 KB)


val: Scanning /content/smokedata/labels/val.cache... 0 images, 180 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:00<?, ?it/s]

WARNING ⚠️ No labels found in /content/smokedata/labels/val.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:03<00:00,  3.56it/s]
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:491: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:526: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:526: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_

                   all        180          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


...

WARNING ⚠️ No labels found in /content/smokedata/labels/val.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:14<00:00,  1.57it/s]
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:491: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:526: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:526: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_

                   all        180          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:655: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index


Speed: 1.5ms preprocess, 76.7ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/baseline_transformer2



# 3. Улучшение бейзлайна
# a. Формулировка гипотез
# - Применение аугментаций (flip, HSV, scale)
# - Использование другой модели (YOLOv8m)
# - Настройка learning rate

# b. Проверка гипотез

In [15]:
model_aug = YOLO('yolov8m.pt')
model_aug.train(data='/content/smokedata/data.yaml', epochs=3, imgsz=640, batch=8, name='augmented',
                augment=True, lr0=0.005)

100%|██████████| 49.7M/49.7M [00:00<00:00, 296MB/s]


Ultralytics 8.3.114 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/smokedata/data.yaml, epochs=3, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=augmented, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_wid

train: Scanning /content/smokedata/labels/train.cache... 716 images, 5 backgrounds, 381 corrupt: 100%|██████████| 716/716 [00:00<?, ?it/s]

train: /content/smokedata/images/train/notsmoking_0001.jpg: ignoring corrupt image/label: Label class 32 exceeds dataset class count 2. Possible class labels are 0-1
train: /content/smokedata/images/train/notsmoking_0010.jpg: ignoring corrupt image/label: Label class 27 exceeds dataset class count 2. Possible class labels are 0-1
train: /content/smokedata/images/train/notsmoking_0011.jpg: ignoring corrupt image/label: Label class 74 exceeds dataset class count 2. Possible class labels are 0-1
train: /content/smokedata/images/train/notsmoking_0019.jpg: ignoring corrupt image/label: Label class 27 exceeds dataset class count 2. Possible class labels are 0-1
train: /content/smokedata/images/train/notsmoking_0022.jpg: ignoring corrupt image/label: Label class 66 exceeds dataset class count 2. Possible class labels are 0-1
train: /content/smokedata/images/train/notsmoking_0025.jpg: ignoring corrupt image/label: Label class 59 exceeds dataset class count 2. Possible class labels are 0-1
trai

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 578.6±318.2 MB/s, size: 75.6 KB)


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:06<00:00,  1.93it/s]
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:491: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:526: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:526: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_m

                   all        180          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:655: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index


Speed: 0.3ms preprocess, 30.4ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/augmented


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([], dtype=int64)
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7be998e281d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        


# e. Оценка качества модели с улучшенным бейзлайном

In [16]:
metrics_aug = model_aug.val()


Ultralytics 8.3.114 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2542.1±770.1 MB/s, size: 81.9 KB)


val: Scanning /content/smokedata/labels/val.cache... 0 images, 180 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:00<?, ?it/s]

WARNING ⚠️ No labels found in /content/smokedata/labels/val.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:11<00:00,  2.02it/s]
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:491: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:526: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:526: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_

                   all        180          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:655: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index


Speed: 2.1ms preprocess, 58.4ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/augmented2


# 4. Имплементация алгоритма машинного обучения
# a. Самостоятельная имплементация простой классификатора на CNN в PyTorch

In [29]:
import os
import shutil

base_dir = '/content/smokedata_raw'
splits = ['Training', 'Validation', 'Testing']
classes = ['smoking', 'nosmoking']

for split in splits:
    split_dir = os.path.join(base_dir, split)
    for cls in classes:
        os.makedirs(os.path.join(split_dir, cls), exist_ok=True)

    for filename in os.listdir(split_dir):
        if filename.endswith(('.jpg', '.png')):
            src = os.path.join(split_dir, filename)

            # Класс определяем по имени файла
            label = 'smoking' if 'smoking' in filename.lower() else 'nosmoking'
            dst = os.path.join(split_dir, label, filename)

            shutil.move(src, dst)

print(" Изображения распределены по классам.")


 Изображения распределены по классам.


In [25]:
import os
import shutil

def organize_images(root_path):
    for split in ['Training', 'Validation', 'Testing']:
        full_path = os.path.join(root_path, split, split)
        smoking_path = os.path.join(full_path, 'smoking')
        nosmoking_path = os.path.join(full_path, 'nosmoking')

        os.makedirs(smoking_path, exist_ok=True)
        os.makedirs(nosmoking_path, exist_ok=True)

        for filename in os.listdir(full_path):
            src = os.path.join(full_path, filename)
            if os.path.isfile(src):
                lower = filename.lower()
                if lower.startswith('smoking'):
                    shutil.move(src, os.path.join(smoking_path, filename))
                elif lower.startswith('nosmoking') or lower.startswith('notsmoking'):
                    shutil.move(src, os.path.join(nosmoking_path, filename))

organize_images('/content/smokedata_raw')




In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_data = datasets.ImageFolder(f'/content/smokedata_raw/Training/Training', transform=transform)
val_data = datasets.ImageFolder(f'/content/smokedata_raw/Validation/Validation', transform=transform)
test_data = datasets.ImageFolder(f'/content/smokedata_raw/Testing/Testing', transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)
test_loader = DataLoader(test_data, batch_size=32)

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 56 * 56, 100),
            nn.ReLU(),
            nn.Linear(100, 2)
        )
    def forward(self, x):
        x = self.conv(x)
        return self.fc(x)

model_custom = SimpleCNN().to('cuda')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_custom.parameters(), lr=0.001)

# Обучение модели

In [27]:
for epoch in range(3):
    model_custom.train()
    for images, labels in train_loader:
        images, labels = images.to('cuda'), labels.to('cuda')
        optimizer.zero_grad()
        outputs = model_custom(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Оценка модели

In [28]:
model_custom.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to('cuda')
        outputs = model_custom(images)
        y_pred.extend(torch.argmax(outputs, dim=1).cpu().numpy())
        y_true.extend(labels.numpy())

print(classification_report(y_true, y_pred, target_names=["non-smoker", "smoker"]))

              precision    recall  f1-score   support

  non-smoker       0.82      0.56      0.67       112
      smoker       0.67      0.88      0.76       112

    accuracy                           0.72       224
   macro avg       0.74      0.72      0.71       224
weighted avg       0.74      0.72      0.71       224

